In [1]:
#!pip install azureml-sdk

In [33]:
import azure.core
from azureml.core import Workspace, Environment,Experiment,ScriptRun
from azureml.core.compute import AmlCompute, ComputeInstance # for creating a compute cluster from python code
from azureml.core.compute import ComputeTarget
from azureml.core import ScriptRunConfig

## Creating Workspace

In [6]:
ws = Workspace.create("demows", resource_group="analytics",
                      subscription_id="a731217c-399f-4a77-9db6-81293ed043d6",
                      create_resource_group=True,
                      location='eastus2')

Deploying StorageAccount with name demowsstoragece5150448bf.
Deploying KeyVault with name demowskeyvault44de8d4abc.
Deployed KeyVault with name demowskeyvault44de8d4abc. Took 28.55 seconds.
Deploying AppInsights with name demowsinsights69ccc697e7.
Deployed AppInsights with name demowsinsights69ccc697e7. Took 74.7 seconds.
Deploying Workspace with name demows.
Deployed Workspace with name demows. Took 43.34 seconds.


In [7]:
#if we want to place this workspace into a local folder
ws.write_config('.azureml')

In [8]:
# see workspace details
ws.from_config()

Workspace.create(name='demows', subscription_id='a731217c-399f-4a77-9db6-81293ed043d6', resource_group='analytics')

In [9]:
# just an try and except block for workspace
#i.e if workspace is alrady present if not create

try:
    ws.write_config('.azureml')
except:
    ws = Workspace.create("demows", resource_group="analytics",
                      subscription_id="a731217c-399f-4a77-9db6-81293ed043d6",
                      create_resource_group=True,
                      location='eastus2')

## Create Compute Cluster

In [16]:
# Creating compute cluster will by default create 2 datastorages
try:
    cluster = ComputeTarget(ws, 'demo-cluster') # if cluster is already present
except:
    # creating a config file for the compute cluster
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2',
                                                  min_nodes=0,
                                                  max_nodes=6,
                                                  idle_seconds_before_scaledown=2400)
    # creating a compute cluster in workspace('ws') with above config
    cluster = ComputeTarget.create(ws,'demo-cluster',config)

## How can we submit code from local machine to azure

In [17]:
# shows how many compute targets are present
ws.compute_targets

{'demo-cluster': AmlCompute(workspace=Workspace.create(name='demows', subscription_id='a731217c-399f-4a77-9db6-81293ed043d6', resource_group='analytics'), name=demo-cluster, id=/subscriptions/a731217c-399f-4a77-9db6-81293ed043d6/resourceGroups/analytics/providers/Microsoft.MachineLearningServices/workspaces/demows/computes/demo-cluster, type=AmlCompute, provisioning_state=Succeeded, location=eastus2, tags={})}

In [19]:
# will show all datastorage
ws.datastores

{'workspaceworkingdirectory': {
   "name": "workspaceworkingdirectory",
   "container_name": "code-391ff5ac-6576-460f-ba4d-7e03433c68b6",
   "account_name": "demowsstoragece5150448bf",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspaceartifactstore': {
   "name": "workspaceartifactstore",
   "container_name": "azureml",
   "account_name": "demowsstoragece5150448bf",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspaceblobstore': {
   "name": "workspaceblobstore",
   "container_name": "azureml-blobstore-b0d997a6-518f-4486-91e5-d63d731cf8ff",
   "account_name": "demowsstoragece5150448bf",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspacefilestore': {
   "name": "workspacefilestore",
   "container_name": "azureml-filestore-b0d997a6-518f-4486-91e5-d63d731cf8ff",
   "account_name": "demowsstoragece5150448bf",
   "protocol": "https",
   "endpoint": "core.windows.net"
 }}

### Creating an Experiment/Job

In [20]:
exp = Experiment(ws,"demoexp")

In [22]:
# shows this recently created experiment is linked to perticular workspace
# note: here experiment is mean by job
exp

Name,Workspace,Report Page,Docs Page
demoexp,demows,Link to Azure Machine Learning studio,Link to Documentation


#### Creating a environment

In [38]:
# Just like in our local machine we create an environment we also need to create same in azure, to run the job.

env = Environment.from_conda_specification(name="virtualenv", file_path='./envfile.yaml')
# env = "virtualenv"

ParserError: while parsing a block collection
  in "./envfile.yaml", line 6, column 3
expected <block end>, but found '?'
  in "./envfile.yaml", line 11, column 3

In [36]:
pythonconfig = ScriptRunConfig(source_directory="./",script="Train.py",compute_target= "demo-cluster",environment="virtualenv")
execution = exp.submit(pythonconfig)

AttributeError: 'str' object has no attribute 'docker'